*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit

# Fleur code

Input generator code creates inp.xml input file from a simplified input file. To use this code via AiiDA, it is needed to implement a corresponding python class that contaings all the logic needed for the use of this code. The class `FleurCalculation` and it's parser already exist in AiiDA-FLEUR plugin. In this tutorial we will learn how to use it in AiiDA inteface.

First things first - let us get familiar with inputs of the FleurinputgenCalculation.

## Inputs of the Fleur calculation

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Fleur code| yes |
|fleurinpdata | FleurinpData | Object representing inp.xml| no |
|parent_folder | RemoteData | Remote folder of another calculation| no |
|settings | Dict | special settings | no |

**code** is a code object representing Fleur code and stored in the database, it is the only required input. However, it is not required to specife **code** only: one *must* provide one of supported input configurations:

* **code** + **fleurinpdata** : start a calculation from scratch with given inp.xml
* **code** + **parent_folder** : start a calculation using inp.xml and charge density from the previous calculation
* **code** + **fleurinpdata** + **parent_folder** : start a calculation using inp.xml from given FleurinpData and charge density from the previous calculation

In this tutorial we are going to use **code** + **fleurinpdata** configuration only.

**settings** is used to modify a list of files to be copied to/from/between calculation folders. We will not cover this in the tutorial.


## Fleur code submission

First of all, we need to import `FleurinputgenCalculation` class.

In [ ]:
from aiida_fleur.calculation.fleur import FleurinputgenCalculation

and prepare input nodes. Configured and ready-to-use inpgen code object is already stored in your database under the number pk=xxx. Now you can simple load it from the database:

In [ ]:
code = load_node(xxx)

We are going to use a FleurinpData that you have created in tutorial 4. Please, find a pk of an Fe monolayer structure and load in via the same command:

In [ ]:
fleurinpdata = load_node(xxx)

In [ ]:
res = submit(FleurinputgenCalculation, code=code, fleurinpdata=fleurinpdata)

You can explore it's outputs: